# Encrypted classification (with Bob and Alice)
## This project uses Pytorch and Pysyft to perform encrypted classification, distributed on two workers, on images from the MNIST database.
A simple (so that can be easily trained on CPU) fully connected network with one hidden layer is built using Pytorch, and trained on the MNIST database. Both the data and the trained model are then encrypted using a private additive sharing encryption scheme, and sent to our two workers Bob and Alice for inference.

Import dependencies

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import syft

W0812 12:56:44.243318  3484 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\apopesc2\AppData\Local\Continuum\anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0812 12:56:44.291685  3484 deprecation_wrapper.py:119] From C:\Users\apopesc2\AppData\Local\Continuum\anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Hook our workers and define the `crypto_provider`

In [2]:
hook = syft.TorchHook(torch) 
bob = syft.VirtualWorker(hook, id="bob")
alice = syft.VirtualWorker(hook, id="alice")
crypto_provider = syft.VirtualWorker(hook, id="crypto_provider")

Define a class of hyperparameters and instantiate

In [3]:
class Hyperparams():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 100
        self.epochs = 5
        self.lr = 0.001

args = Hyperparams()

Download, process MNIST data, and build the training and testing dataloaders

In [4]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True)

Define the model: fully conected network with one hidden layer and Relu activation, going from the input size of 28x28 to the hidden layer size 512 and to the output size of 10 digits

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

Define the training function

In [6]:
def train(args, model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        output = F.log_softmax(output, dim=1)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Epoch: {} -> {:.0f}% done...'.format(
                epoch, 100. * batch_idx / len(train_loader)))
                

Instantiate and train the model

In [7]:
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)

for epoch in range(1, args.epochs + 1):
    train(args, model, train_loader, optimizer, epoch)

Epoch: 1 -> 0% done...
Epoch: 1 -> 11% done...
Epoch: 1 -> 21% done...
Epoch: 1 -> 32% done...
Epoch: 1 -> 43% done...
Epoch: 1 -> 53% done...
Epoch: 1 -> 64% done...
Epoch: 1 -> 75% done...
Epoch: 1 -> 85% done...
Epoch: 1 -> 96% done...
Epoch: 2 -> 0% done...
Epoch: 2 -> 11% done...
Epoch: 2 -> 21% done...
Epoch: 2 -> 32% done...
Epoch: 2 -> 43% done...
Epoch: 2 -> 53% done...
Epoch: 2 -> 64% done...
Epoch: 2 -> 75% done...
Epoch: 2 -> 85% done...
Epoch: 2 -> 96% done...
Epoch: 3 -> 0% done...
Epoch: 3 -> 11% done...
Epoch: 3 -> 21% done...
Epoch: 3 -> 32% done...
Epoch: 3 -> 43% done...
Epoch: 3 -> 53% done...
Epoch: 3 -> 64% done...
Epoch: 3 -> 75% done...
Epoch: 3 -> 85% done...
Epoch: 3 -> 96% done...
Epoch: 4 -> 0% done...
Epoch: 4 -> 11% done...
Epoch: 4 -> 21% done...
Epoch: 4 -> 32% done...
Epoch: 4 -> 43% done...
Epoch: 4 -> 53% done...
Epoch: 4 -> 64% done...
Epoch: 4 -> 75% done...
Epoch: 4 -> 85% done...
Epoch: 4 -> 96% done...
Epoch: 5 -> 0% done...
Epoch: 5 -> 11% done.

Send the encrypted model and data to Bob and Alice

In [8]:
model.fix_precision().share(alice, bob, crypto_provider=crypto_provider)

private_test_loader = []
for data, target in test_loader:
    private_test_loader.append((
        data.fix_prec().share(alice, bob, crypto_provider=crypto_provider),
        target.fix_prec().share(alice, bob, crypto_provider=crypto_provider)))
    

In [9]:
def test(args, model, test_loader):
    model.eval()
    n_correct_priv = 0
    n_total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            pred = output.argmax(dim=1) 
            n_total += args.test_batch_size
            
            # get and decrypt data from workers (inspired from Theo Ryffel's Openmined post)
            n_correct_priv += pred.eq(target.view_as(pred)).sum()
            n_correct = n_correct_priv.copy().get().float_precision().long().item()
    
            print('Test accuracy: {:.0f}%'.format(100. * n_correct / n_total))
                

In [10]:
test(args, model, private_test_loader)

Test accuracy: 97%
Test accuracy: 97%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 97%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accuracy: 98%
Test accurac

The project shows that it is posible to obtaine a good accuracy by performing encrypted inference on privately held data with an encrypted trained model.